In [1]:
import os
import pandas as pd
import numpy as np
from GAE.custom_layers import ConvTypes
from GAE.LossManager import LossTypes, LossManager
import scikit_posthocs as sp
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from GAE.utils import *
import operator
import pickle 
from matplotlib.lines import Line2D

from FramsticksLib import *
import subprocess

In [7]:
## multi_layers for gcs
# path_models = "/home/adam/thesis/VGAE/experiments/experiment_multi_layers/models/"
# path_configs = "/home/adam/thesis/VGAE/experiments/experiment_multi_layers/models/No/configs/"
# model_names = "gcs_diff_layers"

## base_all 
# path_models = "/home/adam/thesis/VGAE/experiments/experiment_base/models/"
# path_configs = "/home/adam/thesis/VGAE/experiments/experiment_base/models/No/configs/"
# model_names = "base_all_convtypes"

## joints
# path_models = "/home/adam/thesis/VGAE/experiments/experiment_base/models/"
# path_configs = "/home/adam/thesis/VGAE/experiments/experiment_base/models/joints/configs/"
# model_names = "base_joints"


#### LOCALITY LOSS:
#all
# path_models = "/home/adam/thesis/VGAE/experiments/experiment_base/models/"
# path_configs = "/home/adam/thesis/VGAE/experiments/experiment_base/configs_all/"
# model_names = "locality_all_convtypes"


#parts
# path_models = "/home/adam/thesis/VGAE/experiments/experiment_base/models/"
# path_configs = "/home/adam/thesis/VGAE/experiments/experiment_base/models/joints/configs/"
# model_names = "locality_joints_convtypes"

#fitness
# path_models = "/home/adam/thesis/VGAE/experiments/experiment_base/models/"
# path_configs = "/home/adam/thesis/VGAE/experiments/experiment_base/models/fitness/configs/"
# model_names = "locality_fitness_convtypes"

#dissim 
# path_models = "/home/adam/thesis/VGAE/experiments/experiment_base/models/"
# path_configs = "/home/adam/thesis/VGAE/experiments/experiment_base/models/dissim/configs/"
# model_names = "locality_dissim_convtypes"


##VERTPOS
path_models = "/home/adam/thesis/VGAE/experiments/experiment_vertpos/models_vertpos/"
path_configs = "/home/adam/thesis/VGAE/experiments/experiment_vertpos/models_vertpos/configs/"
model_names = "vertpos_all"


In [8]:
pathframs = "/home/adam/Framsticks/Framsticks50rc19"
framsmanager = FramsManager(pathframs)

framsLib = FramsticksLib(pathframs,None,["/home/adam/thesis/VGAE/framspy/eval-allcriteria_new.sim"])


Using Framsticks version: 5.0rc19.1
Home (writable) dir     : /home/adam/Framsticks/Framsticks50rc19/data
Resources dir           : /home/adam/Framsticks/Framsticks50rc19/data

Using Framsticks version: 5.0rc19.1
Home (writable) dir     : /home/adam/Framsticks/Framsticks50rc19/data
Resources dir           : /home/adam/Framsticks/Framsticks50rc19/data

Available objects: ['CheckpointEvent', 'Collision', 'CrCollision', 'Creature', 'CreatureSettings', 'CreatureSignals', 'CreatureSnapshot', 'Dictionary', 'ExpProperties', 'ExpState', 'ExtValue', 'File', 'FunctionReference', 'GenMan', 'GenManStats', 'GenePool', 'GenePools', 'Geno', 'Genotype', 'Interface', 'Joint', 'Loader', 'Math', 'MechJoint', 'MechPart', 'MessageCatcher', 'Model', 'ModelGeometry', 'ModelSymmetry', 'Neuro', 'NeuroClass', 'NeuroClassLibrary', 'NeuroDef', 'NeuroSignals', 'NeuronsSimEnabled', 'ODE', 'Orient', 'Part', 'Population', 'Populations', 'Ref', 'Signal', 'SignalView', 'SimilMeasure', 'SimilMeasureDistribution', 'Simil

In [9]:
def get_convType(name):
    if name =="gcnconv":
        return "GCNConv"
    elif name =="armaconv":
        return "ARMAConv"
    elif name =="eccconv":
        return "ECCConv"
    elif name =="gatconv":
        return "GATConv"
    elif name =="gcsconv":
        return "GCSConv"
    
def get_Loss(name):
    if name =="joints":
        return "joints"
    elif name =="parts":
        return "parts"
    elif name =="fitness":
        return "fitness"
    elif name =="dissim":
        return "dissim"
    elif name =="None":
        return "No"

In [10]:
def load_losses(path):
    losses_all_test = []
    losses_all_train = []
    try:
        with open("{0}/losses_test.npy".format(path), "rb") as infile: 
            losses_all_test = np.load(infile).tolist()
        with open("{0}/losses_train.npy".format(path), "rb") as infile: 
            losses_all_train = np.load(infile).tolist()
#         print("Losses loaded successfully")
    except Exception as e:
        print("Error while loading Losses ", e)
    return losses_all_train, losses_all_test

In [11]:
columns = ["path","name","counter","loss_type","ae_type","convtype","latentdim","nhidden","convenc","denseenc","densedeca","convdecx","densedecx","path_model","losses_all_train", "losses_all_test","loss_train","loss_test"]
all_models= pd.DataFrame(columns=columns )
batchsize = 256
learningrate = 0.01
numfeatures = 3
adjsize = 15
counter = 0
for path, subdirs, files in os.walk(path_configs):
#     print(files)
    for file_name in files:
        if "vTrue" in file_name:
            ae_type = "VGAE"
        else:
            ae_type = "GAE"
        splitted = file_name.split("_")
        
        convtype = get_convType(splitted[1])
        latentdim = splitted[2]
        nhidden = splitted[3]
        convenc = splitted[4]
        denseenc = splitted[5]
        densedeca = splitted[6]
        convdecx = splitted[7]
        densedecx = splitted[8]
        loss_type = get_Loss(splitted[9])
        

        for trainid in range(1,11,1):
            path_base = (
                loss_type+
                "/"+ae_type +
                "/numfeatures"+str(numfeatures) +
                "/adjsize"+str(adjsize) + 
                "/batchsize"+str(batchsize) +
                "/latentdim"+str(latentdim)+
                "/nhidden"+str(nhidden)+
                "/learningrate"+str(learningrate)+
                "/convtype"+str(convtype)+
                "/model_enc_"+str(convenc)+"_"+str(denseenc)+
                 "_deca"+str(densedeca)+
                 "_decx"+str(convdecx)+"_"+str(densedecx)+
                 "_train_id_"+str(trainid)
                )
            path_model = str(path_models)+path_base
            
            losses_all_train, losses_all_test = load_losses(path_model)
            loss_train = losses_all_train[-1][1]
            loss_test = losses_all_test[-1][1]
            name = file_name.split("_")
            f = operator.itemgetter(4,5,6,7)
            sublist = list(f(name))
#             print(sublist)
            new_name = ""
            for e in sublist:
                new_name = new_name+e+"_" 
            name =file_name+str(trainid)
            data_model = [[path_base,name,counter,loss_type,ae_type,convtype,latentdim,nhidden,convenc,denseenc,densedeca,convdecx,densedecx,path_model,losses_all_train, losses_all_test,loss_train,loss_test]]
            data_model = pd.DataFrame(data_model,columns=columns) 
            all_models = all_models.append(data_model, ignore_index=True)
        counter+=1


In [12]:
all_models.shape

(300, 18)

In [13]:
all_models.sort_values(by="loss_test")
print("Done")

Done


In [14]:
def get_models_list_of_losses(all_models):
    current_columns = columns + ['losses_train','losses_test','mean_loss_test','mean_loss_train','std_loss_test','std_loss_train',]
    counters  = set(all_models['counter'])
    finall_df  = pd.DataFrame(columns=current_columns )
    for c in counters:
        counter_models = all_models.loc[all_models['counter']==c]
        losses_train = list(counter_models['loss_train'])
        losses_test = list(counter_models['loss_test'])
        row = all_models.loc[all_models['loss_test']==min(losses_test)].iloc[0]
        row['losses_train'] = losses_train
        row['losses_test']= losses_test
        row['mean_loss_train'] = np.mean(losses_train)
        row['mean_loss_test']= np.mean(losses_test)
        row['std_loss_train'] = np.std(losses_train)
        row['std_loss_test']= np.std(losses_test)
        finall_df = finall_df.append(row, ignore_index=True)
    return finall_df

In [15]:

finall_df = get_models_list_of_losses(all_models)


In [21]:
finall_df.shape

(10, 24)

In [20]:
finall_df = finall_df.loc[finall_df['latentdim'] =="15"]

In [22]:
path_config_folder = "/home/adam/thesis/VGAE/config_evol/"
ranges = [0.0333,0.1,0.333]
paths = []
for indeks,row in finall_df.iterrows():
        c = row
#         print(c['path'])
        for r in ranges:
#             print(c['name'])
            f = open(path_config_folder+c['name']+"_"+str(r), "w")
            f.write(c['path']+ "/")
            f.write("\n"+str(r))
            f.close()


In [15]:
# path_config_folder = "/home/adam/thesis/VGAE/config_evol/"
# ranges = [0.0333,0.1,0.333]
# paths = []
# for l in set(finall_df['latentdim']):
#     for a in set(finall_df['ae_type']):
#         f_df = finall_df.loc[finall_df['latentdim']==l].loc[finall_df['ae_type']==a]
#         c = f_df.loc[f_df['loss_test']==min(f_df['loss_test'])]
#         for r in ranges:
#             print(c.name.values[0])
#             f = open(path_config_folder+f_df.name.values[0]+"_"+str(r), "w")
#             f.write(c['path']+ "/")
#             f.write("\n"+str(r))
#             f.close()
# # paths = list(finall_df['path_model'])[1:-1]

vTrue_gcsconv_15_64_1_2_1_1_2_parts10
vTrue_gcsconv_15_64_1_2_1_1_2_parts10
vTrue_gcsconv_15_64_1_2_1_1_2_parts10
vFalse_gcsconv_15_64_1_2_1_1_2_fitness8
vFalse_gcsconv_15_64_1_2_1_1_2_fitness8
vFalse_gcsconv_15_64_1_2_1_1_2_fitness8


In [89]:
paths = []
for l in set(finall_df['latentdim']):
    for a in set(finall_df['ae_type']):
        f_df = finall_df.loc[finall_df['latentdim']==l].loc[finall_df['ae_type']==a]
        c = f_df.loc[f_df['loss_test']==min(f_df['loss_test'])]
        paths.append(c['path_model'].values[0])

In [90]:
# paths = list(finall_df.loc[finall_df['latentdim']=='15']['path_model'].values)
paths

['/home/adam/thesis/VGAE/experiments/experiment_vertpos/models_vertpos/fitness/GAE/numfeatures3/adjsize15/batchsize256/latentdim10/nhidden64/learningrate0.01/convtypeGCSConv/model_enc_2_2_deca1_decx1_2_train_id_10',
 '/home/adam/thesis/VGAE/experiments/experiment_vertpos/models_vertpos/dissim/VGAE/numfeatures3/adjsize15/batchsize256/latentdim10/nhidden64/learningrate0.01/convtypeGCSConv/model_enc_2_2_deca1_decx1_2_train_id_10',
 '/home/adam/thesis/VGAE/experiments/experiment_vertpos/models_vertpos/dissim/GAE/numfeatures3/adjsize15/batchsize256/latentdim15/nhidden64/learningrate0.01/convtypeGCSConv/model_enc_2_2_deca1_decx1_2_train_id_7',
 '/home/adam/thesis/VGAE/experiments/experiment_vertpos/models_vertpos/dissim/VGAE/numfeatures3/adjsize15/batchsize256/latentdim15/nhidden64/learningrate0.01/convtypeGCSConv/model_enc_2_2_deca1_decx1_2_train_id_1',
 '/home/adam/thesis/VGAE/experiments/experiment_vertpos/models_vertpos/fitness/GAE/numfeatures3/adjsize15/batchsize256/latentdim3/nhidden64

In [91]:
cat /home/adam/thesis/VGAE/experiments/experiment_vertpos/models_vertpos/No/VGAE/numfeatures3/adjsize15/batchsize256/latentdim3/nhidden64/learningrate0.01/convtypeGCSConv/model_enc_2_2_deca1_decx1_2_train_id_7/args.txt

/home/inf131778/Framsticks50rc19/
graphs/gen_vertpos/
models_vertpos/
256
15
3
3
64
2
2
1
1
2
0.01
100
gcsconv
True
No
7


In [92]:
paths

['/home/adam/thesis/VGAE/experiments/experiment_vertpos/models_vertpos/fitness/GAE/numfeatures3/adjsize15/batchsize256/latentdim10/nhidden64/learningrate0.01/convtypeGCSConv/model_enc_2_2_deca1_decx1_2_train_id_10',
 '/home/adam/thesis/VGAE/experiments/experiment_vertpos/models_vertpos/dissim/VGAE/numfeatures3/adjsize15/batchsize256/latentdim10/nhidden64/learningrate0.01/convtypeGCSConv/model_enc_2_2_deca1_decx1_2_train_id_10',
 '/home/adam/thesis/VGAE/experiments/experiment_vertpos/models_vertpos/dissim/GAE/numfeatures3/adjsize15/batchsize256/latentdim15/nhidden64/learningrate0.01/convtypeGCSConv/model_enc_2_2_deca1_decx1_2_train_id_7',
 '/home/adam/thesis/VGAE/experiments/experiment_vertpos/models_vertpos/dissim/VGAE/numfeatures3/adjsize15/batchsize256/latentdim15/nhidden64/learningrate0.01/convtypeGCSConv/model_enc_2_2_deca1_decx1_2_train_id_1',
 '/home/adam/thesis/VGAE/experiments/experiment_vertpos/models_vertpos/fitness/GAE/numfeatures3/adjsize15/batchsize256/latentdim3/nhidden64

In [93]:
# path = finall_df['path_model'][0]+"/args.txt"
# print(path)
# !./run_locally.sh $(cat $path)

# for p in paths:
#     path = p+"/args.txt"
# #     print(path)
#     !./run_locally.sh $(cat $path)

In [94]:
# x = "No"
# ![if $x=='no'];then
#     echo $x
# !fi

In [ ]:
# ranges = [[0,0.00001],[0,0.0001],[0,0.001],[0,0.01],[0,1],[0,1],[0,10],[0,100],[0,1000]]
ranges = [0.0001,0.000333,0.001,0.00333,0.01,0.0333,0.1,0.333,1,3.333]
np.set_printoptions(suppress=True)

max_e = 1000

def get_fit_for_path(path,base =False):   
    pickle_off = open (path, "rb")
    emp = pickle.load(pickle_off)
    
    if base:
        frams_gen,z_all = gen_f0_from_df(emp,max_elements=max_e)
    else:
        frams_gen,z_all = gen_f0_from_df(emp.iloc[:,3:],max_elements=max_e)
    gen_correct = []
    z_correct = []
    for i in range(len(frams_gen)):
#         print(i)
        gen = frams_gen[i]
        z = z_all[i]
        if type(z) == np.ndarray:
            z = z.tolist()
        else:
            z = z.numpy().tolist()
#         print(type(z))
#                 print("check_consistency_for_gen")
        gen = framsmanager.check_consistency_for_gen(gen)
        if gen is not None:
#                     print("reduce_joint_length_for_gen")
            gen = framsmanager.reduce_joint_length_for_gen(gen)
        if gen is not None:
            gen_correct.append(gen)
            z_correct.append(z)
            
    print((len(gen_correct)))
    fit_all = []
    n_parts_all = []
    n_joints_all = []
    c = framsLib.evaluate(gen_correct)
#     print(c)
    for i in range(len(c)):
        fit_all.append(c[i]['evaluations']['']['vertpos'])
        n_parts_all.append(c[i]['evaluations']['']['numparts'])
        n_joints_all.append(c[i]['evaluations']['']['numjoints'])        
    return fit_all,n_parts_all,n_joints_all,z_correct

for p in paths:# finall_df['path_model'].iloc[1:]:
#     p= finall_df['path_model'][0]
    print(p)
    base = []
    val_for_r=[]
    for r in ranges:
        path = p+"/z_random_{0}_{1}.pkl".format(-r,r)  
#         path = p+"/z_training_{0}_{1}.pkl".format(-r,r)
        if len(base)==0:
            base = [get_fit_for_path(path,base=True)]
            
        val_for_r.append([get_fit_for_path(path,base=False)])

    pd_val_all = pd.DataFrame(val_for_r,index=ranges)
    pd_base = pd.DataFrame(base)
    pd_val_all.to_csv(p+"/pd_random_val_all.csv")
    pd_base.to_csv(p+"/pd_random_base.csv")
    
    base = []
    val_for_r=[]
    for r in ranges:
#         path = p+"/z_random_{0}_{1}.pkl".format(-r,r)  
        path = p+"/z_training_{0}_{1}.pkl".format(-r,r)
        if len(base)==0:
            base = [get_fit_for_path(path,base=True)]
            
        val_for_r.append([get_fit_for_path(path,base=False)])

    pd_val_all = pd.DataFrame(val_for_r,index=ranges)
    pd_base = pd.DataFrame(base)
    pd_val_all.to_csv(p+"/pd_val_all.csv")
    pd_base.to_csv(p+"/pd_base.csv")
#     break

/home/adam/thesis/VGAE/experiments/experiment_vertpos/models_vertpos/fitness/GAE/numfeatures3/adjsize15/batchsize256/latentdim10/nhidden64/learningrate0.01/convtypeGCSConv/model_enc_2_2_deca1_decx1_2_train_id_10
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000


Exception ignored in: <function ExtValue.__del__ at 0x7f5fffcd55f0>
Traceback (most recent call last):
  File "/home/adam/thesis/VGAE/framspy/frams.py", line 53, in __del__
    c_api.extFree(self.__ptr)
  File "/home/adam/thesis/VGAE/framspy/frams.py", line 172, in __getattr__
    return self.__dict__[key]
KeyError: ('_ExtValue__ptr',)


1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
/home/adam/thesis/VGAE/experiments/experiment_vertpos/models_vertpos/dissim/VGAE/numfeatures3/adjsize15/batchsize256/latentdim10/nhidden64/learningrate0.01/convtypeGCSConv/model_enc_2_2_deca1_decx1_2_train_id_10
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
/home/adam/thesis/VGAE/experiments/experiment_vertpos/models_vertpos/dissim/GAE/numfeatures3/adjsize15/batchsize256/latentdim15/nhidden64/learningrate0.01/convtypeGCSConv/model_enc_2_2_deca1_decx1_2_train_id_7
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
/home/adam/thesis/VGAE/experiments/experiment_vertpos/models_vertpos/dissim/VGAE/numfeatures3/adjsize15/batchsize256/latentdim15/nhidden64/learningrate0.01/convtypeGCSConv/model_enc_2_2_deca1_decx1_2_train_id_1
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
10